<a href="https://colab.research.google.com/github/kkrusere/youTube-comments-Analyzer/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb

wget -N https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/118.0.5993.70/linux64/chromedriver-linux64.zip -P /tmp/
unzip -o /tmp/chromedriver-linux64.zip -d /tmp/
chmod +x /tmp/chromedriver-linux64/chromedriver
mv /tmp/chromedriver-linux64/chromedriver /usr/local/bin/chromedriver
pip install selenium chromedriver_autoinstaller

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
51 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
w

In [2]:
import os
from google.colab import drive

In [3]:
#mounting google drive
drive.mount('/content/drive')

########################################

#changing the working directory
os.chdir("/content/drive/MyDrive/EV NLP Data")

!pwd


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/EV NLP Data


In [4]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')


from selenium import webdriver
import chromedriver_autoinstaller
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time


chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chromedriver_autoinstaller.install()
driver = webdriver.Chrome(options=chrome_options)


from bs4 import BeautifulSoup
import json

In [5]:
# YouTube video URL
video_url = "https://www.youtube.com/watch?v=-cJ1fSX3Nec"

In [6]:
try:
    wait = WebDriverWait(driver, 5)

    # Open the YouTube video
    driver.get(video_url)

    wait.until(EC.visibility_of_element_located((By.TAG_NAME, 'body')))

    # Scroll down to load comments
    last_height = 0
    while True:
        # Scroll to the end of the page
        #driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
        # Scroll using page-manager element's scroll height
        driver.execute_script("window.scrollTo(0, document.getElementById('page-manager').scrollHeight);")
        time.sleep(5)  # Adjust sleep time as needed

        # Get the current page height
        new_height = driver.execute_script("return document.getElementById('page-manager').scrollHeight")

        # Break the loop if no more content is loaded
        if new_height == last_height:
            break

        last_height = new_height

    # Wait for the comments section to be visible
    comments_section = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "ytd-comments#comments.style-scope.ytd-watch-flexy")))

    # Get the HTML content of the comments section
    comments_html = comments_section.get_attribute('outerHTML')

finally:
    # Close the WebDriver session
    driver.quit()



In [7]:
soup = BeautifulSoup(comments_html, 'html.parser')

# Find the span element with the specified class
comment_count_span = soup.find('span', class_='style-scope yt-formatted-string')

# Extract the text content of the span element
comment_count = comment_count_span.text.strip()

# Print or use the comment count
print("Comment Count:", comment_count)

Comment Count: 9,327


In [8]:
# Find all occurrences of the ytd-comment-thread-renderer element
comment_thread_renderers = soup.find_all('ytd-comment-thread-renderer', class_='style-scope ytd-item-section-renderer')

# Count the number of occurrences
comment_thread_count = len(comment_thread_renderers)

# Print or use the comment thread count
print("Number of ytd-comment-thread-renderer elements:", comment_thread_count)

Number of ytd-comment-thread-renderer elements: 957


In [9]:
# Function to save comments data to a JSON file
def save_comments_to_json(comments, filename = 'youtube_comments.json'):
    with open(filename, 'w') as json_file:
        json.dump(comments, json_file, indent=4)

def load_comments_from_json(filename = 'youtube_comments.json'):
    with open(filename, 'r') as json_file:
        comments = json.load(json_file)
    return comments

In [10]:
comments = []
comments_data = load_comments_from_json()
# Iterate through each comment thread renderer
for comment_thread_renderer in comment_thread_renderers:

    # Extracting the comment text
    comment_text_element = comment_thread_renderer.find('yt-attributed-string', id='content-text')
    comment_text = comment_text_element.get_text(strip=True) if comment_text_element else None

    # Extracting the number of likes
    like_count_element = comment_thread_renderer.find('span', class_='style-scope ytd-comment-engagement-bar')
    like_count = like_count_element.get_text(strip=True) if like_count_element else None

    # Extracting the number of replies
    reply_count_element = comment_thread_renderer.find('ytd-button-renderer', id='more-replies')
    reply_count = reply_count_element.get_text(strip=True) if reply_count_element else None

    comments.append(comment_text)

    print("Comment Text:", comment_text)
    print("Like Count:", like_count)
    print("Reply Count:", reply_count)

    print("\n\n\n")


    comments_data.append(
        {
        "comment_text": comment_text,
        "like_count": like_count,
        "reply_count": reply_count

        }
    )


save_comments_to_json(comments = comments_data)

Streaming output truncated to the last 5000 lines.
Like Count: 
Reply Count: None




Comment Text: Well,  this is what happens when you try to force people into buying a product that they cannot afford and they cannot supply power for. Rushing technology along before it is totally cooked, without the electrical grid to support it
Like Count: 
Reply Count: None




Comment Text: Mary Barra stated that common people are not her core customers. The auto industry also knows that the government and states will give them tax abatements and bailouts when automakers make poor decisions. The automakers were told by their consumers that we didn't want electric vehicles. But they listened to their donors and forced it on us to the point that we didn't buy into it even as the government was funding them at every corner.
Like Count: 38
Reply Count: 2 replies




Comment Text: We want reliable cars that will get us from point A to point B without hassle at an affordable overall cost.
Like Count: 12

In [11]:
len(load_comments_from_json())

16477

In [4]:
from google.colab import userdata
api_key = userdata.get('YouTubeAPI_key')

In [7]:
import googleapiclient.discovery
import datetime

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

youtube = googleapiclient.discovery.build("youtube", "v3", developerKey=api_key)


In [8]:
def get_video_id(video_url):
  video_id = video_url.split('=')[1]

  return video_id

In [10]:
# Request video details
response = youtube.videos().list(
    part="snippet,statistics",
    id=get_video_id(video_url)
).execute()

# Extract required information
video = response["items"][0]
title = video["snippet"]["title"]
thumbnail_url = video["snippet"]["thumbnails"]["maxres"]["url"]
view_count = video["statistics"]["viewCount"]
like_count = video["statistics"]["likeCount"]
commentCount = video["statistics"]["commentCount"]
date_posted = video["snippet"]["publishedAt"]
description = video["snippet"]["description"]


date_str = date_posted
date_object = datetime.datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%SZ")

# Format the date object
date_posted = date_object.strftime("%Y-%m-%d %H:%M:%S")


# Split the description by newlines
paragraphs = description.split("\n")

# Get the first paragraph
first_paragraph = paragraphs[0]

# Generate HTML to display the thumbnail
thumbnail_html = f'<img src="{thumbnail_url}" alt="Thumbnail">'

# Print the information
print("Title:", title)
print("Thumbnail URL:", thumbnail_html)
print("View count:", view_count)
print("Like count:", like_count)
print("Number of Comments:", commentCount)
print("Date posted:", date_posted)
print("Description:", first_paragraph)


Title: Why EVs Are Piling Up At Dealerships In The U.S.
Thumbnail URL: <img src="https://i.ytimg.com/vi/cZlsZwcIgpc/maxresdefault.jpg" alt="Thumbnail">
View count: 4492458
Like count: 36502
Number of Comments: 23211
Date posted: 2023-10-16 16:26:47
Description: In August 2023, it took about twice as long to sell an EV in the U.S. as it did the previous January. Prices of EVs are down 22% year-over-year and that's mainly driven by Tesla. About two thirds of EVs sold are Elon Musk's brand. Companies like Ford have ramped up hybrid production as demand has leveled off. While slightly more than half of consumers say EVs are the future and will eventually replace Internal Combustion Engines, less than a third of dealers say so. This all comes at a time when investments in EVs are more than ever. So what's really going on? Watch the video to learn more.


In [16]:
# Search for videos using the YouTube API
def search_videos(query, youtube = youtube):

  # Request video search
  response = youtube.search().list(
      q=query,
      type="video",
      part="id,snippet",
      maxResults=10
  ).execute()

  # Extract video IDs from search results
  video_ids = []
  for item in response["items"]:
    video_ids.append(item["id"]["videoId"])

  return video_ids

In [17]:
def get_video_info(video_id):
  # Request video details
  response = youtube.videos().list(
      part="snippet,statistics",
      id=video_id
  ).execute()

  # Extract required information
  video = response["items"][0]
  title = video["snippet"]["title"]
  view_count = video["statistics"]["viewCount"]
  like_count = video["statistics"]["likeCount"]
  commentCount = video["statistics"]["commentCount"]
  date_posted = video["snippet"]["publishedAt"]
  description = video["snippet"]["description"]


  date_str = date_posted
  date_object = datetime.datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%SZ")

  # Format the date object
  date_posted = date_object.strftime("%Y-%m-%d %H:%M:%S")


  # Split the description by newlines
  paragraphs = description.split("\n")

  # Get the first paragraph
  first_paragraph = paragraphs[0]

  extracted_video_url = f"https://www.youtube.com/watch?v={video_id}"


  # Print the information
  print("Title:", title)
  print("Video URL:", extracted_video_url)
  print("View count:", view_count)
  print("Like count:", like_count)
  print("Number of Comments:", commentCount)
  print("Date posted:", date_posted)
  print("Description:", first_paragraph)
  print("\n\n\n")


In [18]:
search_query = "Electric Vehicles"
video_ids = search_videos(search_query)


for video_id in video_ids:
  get_video_info(video_id)

Title: Should I Get An Electric Car?
Video URL: https://www.youtube.com/watch?v=vXjs17ydRgI
View count: 40707
Like count: 2661
Number of Comments: 574
Date posted: 2024-04-10 15:31:46
Description: Check out Insectarium on PBS Terra: https://www.youtube.com/watch?v=jowk_A71-EU




Title: Used electric cars. The FIVE golden buying rules | Electrifying
Video URL: https://www.youtube.com/watch?v=aIYyycWQc34
View count: 13639
Like count: 682
Number of Comments: 96
Date posted: 2024-04-09 15:30:00
Description: #usedelectric #buyingcars #electriccars




Title: Are used EVs a rip-off?!
Video URL: https://www.youtube.com/watch?v=Ur47okU3eUk
View count: 350718
Like count: 13192
Number of Comments: 1927
Date posted: 2024-04-10 10:05:09
Description: Sell your car for free with Carwow: https://bit.ly/Sell-Your-Car-For-Free-1004




Title: Why Hybrids Are Beating EVs In The U.S.
Video URL: https://www.youtube.com/watch?v=R_J_T7TWbXY
View count: 633530
Like count: 9340
Number of Comments: 2979
Date 